In [0]:
#!pip install psycopg2-binary

In [0]:
#!git clone https://github.com/labs15-baseball-pitch-predictor/RC-v1.1--Data-Cleaning-and-Feature-Engineering.git

In [0]:
import pandas as pd
import numpy as np
import random
import pickle
import gc
import psycopg2
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## Load in the pre-pitcher preprocessed_df



In [0]:
#load in pickled preprocessed df

df1 = pd.read_pickle("/content/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/pre_pitcher_pre_processed_df_part1.pkl", compression='zip')
df2 = pd.read_pickle("/content/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/pre_pitcher_pre_processed_df_part2.pkl", compression='zip')

df = pd.concat([df1, df2])

In [73]:
#clear some memory:
del df1
del df2
gc.collect()

28

In [0]:
#make list of pitchers (for filling in Nans in batting stat later for pitcher slot in batting order)

def make_pitcher_list(pre_processed_df):
    #make list of all the pitcher ids in the preprocessed_df
    pitcher_list = df['pitcher'].unique().tolist()
    
    #on rare occassions in extra-inning games, a position player is called in to pitch: 
    #to avoid categorizing him as a pitcher we'll only count pitchers with over 100 pitches:
    for pitcher in pitcher_list:
        if len(df[df['pitcher'] == pitcher]) < 100:
            pitcher_list.remove(pitcher)
    return pitcher_list

pitcher_list = make_pitcher_list(df)

## Choose the pitcher to model

In [0]:
#df.groupby('pitcher').count().sort_values('pitch_type', ascending=False).head(10)

In [76]:
#dictionary mapping top 10 pitchers with most pitches name with pitcher id

pitcher_id_dict = {'Justin Verlander': 434378.0, 'Trevor Bauer': 545333.0, 
                   'Aaron Nola': 605400.0, 'Lance Lynn': 458681.0, 'Gerrit Cole':
                   543037.0, 'Jacob deGrom':594798.0, 'Zack Greinke':425844.0,
                   'Patrick Corbin': 571578.0, 'Max Scherzer': 453286.0,
                   'Kyle Gibson': 502043.0}

#make pitcher_df subset where pitcher == picther_id
# pitcher_id = pitcher_id_dict['Justin Verlander']
# pitcher_id = pitcher_id_dict['Trevor Bauer']
# pitcher_id = pitcher_id_dict['Aaron Nola']
# pitcher_id = pitcher_id_dict['Lance Lynn']
# pitcher_id = pitcher_id_dict['Gerrit Cole']
# pitcher_id = pitcher_id_dict['Jacob deGrom']
# pitcher_id = pitcher_id_dict['Zack Greinke']
pitcher_id = pitcher_id_dict['Patrick Corbin']
# pitcher_id = pitcher_id_dict['Max Scherzer']
# pitcher_id = pitcher_id_dict['Kyle Gibson']

pitcher_df = df[df['pitcher'] == pitcher_id]
pitcher_df.head()

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat
1176,FF,2018-03-29,22960,91.800003,2.2178,6.3667,453568.0,571578.0,11.0,NaN,2018.0,0.5639,1.0993,-0.6422,4.0275,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488671.0,-8.4965,-133.442596,-2.4342,8.6602,29.870899,-18.6679,3.4124,1.6428,NaN,NaN,NaN,91.962997,2216.0,6.539,529410.0,53.963501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,L,L,ARI,COL,B,NaN,Top,180330_021111,Standard,0.0,NaN,True,False,False,False,0,0,0,0,65.776695,29.307116,39.467312,38.118214,0.40704,0.246085,0.221477,24.895977,36.879433,35.422344,28.474577,0.471,0.221649,0.293814,9.327323,37.572254,40.277779,17.708334,0.459517,0.28,0.226667,0.0,0,0,Patrick Corbin,NaN,ball,NaN,4-Seam Fastball,Standard,00,neutral,fastball
1177,FF,2018-03-29,22949,91.900002,2.2893,6.2875,453568.0,571578.0,14.0,NaN,2018.0,0.7047,1.1697,0.1060,1.4088,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488671.0,-7.0477,-133.367096,-8.9738,9.9698,30.138700,-16.4660,3.3972,1.6277,NaN,NaN,NaN,91.573997,2298.0,6.360,529410.0,54.141998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,L,L,ARI,COL,B,NaN,Top,180330_021131,Standard,0.0,NaN,False,True,False,False,0,0,0,0,65.776695,29.307116,39.467312,38.118214,0.40704,0.246085,0.221477,24.895977,36.879433,35.422344,28.474577,0.471,0.221649,0.293814,9.327323,37.572254,40.277779,17.708334,0.459517,0.28,0.226667,0.0,1,0,Patrick Corbin,NaN,ball,NaN,4-Seam Fastball,Standard,10,behind,fastball
1178,FF,2018-03-29,22932,92.599998,2.2268,6.3818,453568.0,571578.0,1.0,NaN,2018.0,0.3072,1.3006,-0.6351,3.0080,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488671.0,-7.9257,-134.451004,-5.5301,5.5154,31.222099,-15.5747,3.1735,1.2750,NaN,NaN,NaN,91.835999,2254.0,6.116,529410.0,54.386101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,L,L,ARI,COL,S,NaN,Top,180330_021149,Standard,0.0,NaN,False,False,False,False,0,1,1,0,65.776695,29.307116,39.467312,38.118214,0.40704,0.246085,0.221477,24.895977,36.879433,35.422344,28.474577,0.471,0.221649,0.293814,9.327323,37.572254,40.277779,17.708334,0.459517,0.28,0.226667,0.0,2,0,Patrick Corbin,NaN,foul,NaN,4-Seam Fastball,Standard,20,behind,fastball
1179,FF,2018-03-29,22923,93.599998,2.1748,6.2880,453568.0,571578.0,7.0,NaN,2018.0,0.4711,1.3943,-0.6142,1.5226,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488671.0,-8.1868,-135.703003,-9.5143,7.6057,30.205700,-13.1636,3.3815,1.5862,NaN,NaN,NaN,93.060997,2301.0,6.214,529410.0,54.287601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,L,L,ARI,COL,S,NaN,Top,180330_021222,Standard,0.0,NaN,False,True,False,False,0,0,0,0,65.776695,29.307116,39.467312,38.118214,0.40704,0.246085,0.221477,24.895977,36.879433,35.422344,28.474577,0.471,0.221649,0.293814,9.

## Add the pitcher prior tendencies (Pitcher Scouting Report)

In [77]:
%%time
#read in 2017 pre_procesed step 1 pickle and filter for pitcher_id
df17 = pd.read_pickle('/content/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/preprocessed_2017.pkl', compression = 'zip')
pitcher_df17 = df17[df17['pitcher'] == pitcher_id]

CPU times: user 2.19 s, sys: 79.9 ms, total: 2.27 s
Wall time: 2.27 s


In [0]:
def get_left_right_pitch_tendencies(pitcher_df):
    #split the df into left hand and right handed batters
    left = pitcher_df[pitcher_df['stand'] == 'L'].copy()
    right = pitcher_df[pitcher_df['stand'] == 'R'].copy()
    
    #assign the normalized value counts for this pitchers pitch types to a dictionary
    overall_left = left['pitch_cat'].value_counts(normalize=True).to_dict()
    overall_right = right['pitch_cat'].value_counts(normalize=True).to_dict()
    
    #initialize empty dict for count categories tendencies
    by_count_left = {}
    by_count_right = {}
    
    #loop over each count category and get the pitchers tendencies and add to the dict
    for cat in pitcher_df['count_cat'].unique().tolist():
        left_subset = left[left['count_cat'] == cat]
        right_subset = right[right['count_cat'] == cat]
        
        by_count_left[cat] = left_subset['pitch_cat'].value_counts(normalize=True).to_dict()
        by_count_right[cat] = right_subset['pitch_cat'].value_counts(normalize=True).to_dict()
        
    return overall_left, overall_right, by_count_left, by_count_right

def make_tendency_features(pitcher_df, overall_left, overall_right, by_count_left, by_count_right):
    #helper functions to vectorize w/ df.apply():
    def get_overall_left_perc(x):
        return overall_left[x] * 100
    def get_overall_right_perc(x):
        return overall_right[x] * 100
    def get_by_count_left_perc(x):
        try:
            return by_count_left[x][pitch_type] * 100
        except KeyError:
            return 0
    def get_by_count_right_perc(x):
        try:
            return by_count_right[x][pitch_type] * 100
        except KeyError:
            return 0
    
    left = pitcher_df[pitcher_df['stand'] == 'L'].copy()
    right = pitcher_df[pitcher_df['stand'] == 'R'].copy()
    
    pitch_types_left = overall_left.keys()
    pitch_types_right = overall_right.keys()
    
    #Left
    for pitch_type in pitch_types_left:
        overall_feature = 'overall_' + pitch_type + '_perc'
        count_cat_feature = 'count_cat_' + pitch_type + '_perc'
      
        left[overall_feature] = pitch_type
        left[overall_feature] = left[overall_feature].apply(get_overall_left_perc)
        left[count_cat_feature] = left['count_cat'].apply(get_by_count_left_perc)
    #Right
    for pitch_type in pitch_types_right:
        overall_feature = 'overall_' + pitch_type + '_perc'
        count_cat_feature = 'count_cat_' + pitch_type + '_perc'
      
        right[overall_feature] = pitch_type
        right[overall_feature] = right[overall_feature].apply(get_overall_right_perc)
        right[count_cat_feature] = right['count_cat'].apply(get_by_count_right_perc)
        
    
    return pd.concat([left,right], sort=False).sort_values(by=['game_date', 'game_pk', 'at_bat_number', 'pitch_number'])

def add_pitcher_scouting_report(pitcher_df, pitcher_df17, start_dates, end_dates):
    df = pd.concat([pitcher_df, pitcher_df17], sort=False)
    
    #initialize empty list to store dfs (concat them together later)
    df_list = []
    
    #iterate over each period
    for i in range(len(start_dates)):
      
        #make the prior and current dfs:
        prior_df = df[df['game_date'] < start_dates[i]]
        current_df = df[(df['game_date'] >= start_dates[i]) & (df['game_date'] <= end_dates[i])].copy()
        
        #get the pitch tendencies from prior:
        overall_left, overall_right, by_count_left, by_count_right = get_left_right_pitch_tendencies(prior_df)
        
        #make the pitch tendencies features on current:
        current_df = make_tendency_features(current_df, overall_left, overall_right, by_count_left, by_count_right)
        
        #append the df to the list
        df_list.append(current_df)
    
    df = pd.concat(df_list, sort=False)
    return df


In [0]:
start_dates = ['2018-03-29', '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01', 
               '2018-09-01', '2019-03-28', '2019-05-01', '2019-06-01', '2019-07-01', 
               '2019-08-01']

end_dates =  ['2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', 
              '2018-10-01', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', 
              '2019-08-31']

In [80]:
%%time
pitcher_df = add_pitcher_scouting_report(pitcher_df, pitcher_df17, start_dates, end_dates)

CPU times: user 1.2 s, sys: 9.99 ms, total: 1.21 s
Wall time: 1.22 s


In [81]:
pitcher_df.head()

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat,overall_breaking_perc,count_cat_breaking_perc,overall_fastball_perc,count_cat_fastball_perc,overall_offspeed_perc,count_cat_offspeed_perc
1176,FF,2018-03-29,22960,91.800003,2.2178,6.3667,453568.0,571578.0,11.0,NaN,2018.0,0.5639,1.0993,-0.6422,4.0275,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488671.0,-8.4965,-133.442596,-2.4342,8.6602,29.870899,-18.6679,3.4124,1.6428,NaN,NaN,NaN,91.962997,2216.0,6.539,529410.0,53.963501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,L,L,ARI,COL,B,NaN,Top,180330_021111,Standard,0.0,NaN,True,False,False,False,0,0,0,0,65.776695,29.307116,39.467312,38.118214,0.40704,0.246085,0.221477,24.895977,36.879433,35.422344,28.474577,0.471,0.221649,0.293814,9.327323,37.572254,40.277779,17.708334,0.459517,0.28,0.226667,0.0,0,0,Patrick Corbin,NaN,ball,NaN,4-Seam Fastball,Standard,00,neutral,fastball,52.143845,53.600000,45.919779,44.400000,1.936376,2.000000
1177,FF,2018-03-29,22949,91.900002,2.2893,6.2875,453568.0,571578.0,14.0,NaN,2018.0,0.7047,1.1697,0.1060,1.4088,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488671.0,-7.0477,-133.367096,-8.9738,9.9698,30.138700,-16.4660,3.3972,1.6277,NaN,NaN,NaN,91.573997,2298.0,6.360,529410.0,54.141998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,L,L,ARI,COL,B,NaN,Top,180330_021131,Standard,0.0,NaN,False,True,False,False,0,0,0,0,65.776695,29.307116,39.467312,38.118214,0.40704,0.246085,0.221477,24.895977,36.879433,35.422344,28.474577,0.471,0.221649,0.293814,9.327323,37.572254,40.277779,17.708334,0.459517,0.28,0.226667,0.0,1,0,Patrick Corbin,NaN,ball,NaN,4-Seam Fastball,Standard,10,behind,fastball,52.143845,37.113402,45.919779,61.855670,1.936376,1.030928
1178,FF,2018-03-29,22932,92.599998,2.2268,6.3818,453568.0,571578.0,1.0,NaN,2018.0,0.3072,1.3006,-0.6351,3.0080,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488671.0,-7.9257,-134.451004,-5.5301,5.5154,31.222099,-15.5747,3.1735,1.2750,NaN,NaN,NaN,91.835999,2254.0,6.116,529410.0,54.386101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,L,L,ARI,COL,S,NaN,Top,180330_021149,Standard,0.0,NaN,False,False,False,False,0,1,1,0,65.776695,29.307116,39.467312,38.118214,0.40704,0.246085,0.221477,24.895977,36.879433,35.422344,28.474577,0.471,0.221649,0.293814,9.327323,37.572254,40.277779,17.708334,0.459517,0.28,0.226667,0.0,2,0,Patrick Corbin,NaN,foul,NaN,4-Seam Fastball,Standard,20,behind,fastball,52.143845,37.113402,45.919779,61.855670,1.936376,1.030928
1179,FF,2018-03-29,22923,93.599998,2.1748,6.2880,453568.0,571578.0,7.0,NaN,2018.0,0.4711,1.3943,-0.6142,1.5226,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488671.0,-8.1868,-135.703003,-9.5143,7.6057,30.205700,-13.1636,3.3815,1.5862,NaN,NaN,NaN,93.060997,2301

## Add Game Features:

#### Batting Order, Game pitch_count, & Trailing Pitch Features

In [0]:
def make_game_batting_order(game_df):
    game_df = game_df.sort_values(by=['at_bat_number', 'pitch_number'])
    all_batters = game_df['batter'].unique().tolist()
    #re-set the at_bat_number for the game to be sequential starting at 1
    at_bat_keys = game_df['at_bat_number'].unique().tolist()
    at_bat_values = range(1, len(at_bat_keys)+1)
    at_bat_map = dict(zip(at_bat_keys, at_bat_values))
    game_df['at_bat_number'] = game_df['at_bat_number'].replace(at_bat_map)
    
    #get the first 9 batter ids
    first_9_batter_subset = game_df[game_df['at_bat_number'] < 10]
    first_9_batters = first_9_batter_subset['batter'].unique().tolist()
    
    #map the batter id to batting order position 1-9
    batting_order_map = dict(zip(first_9_batters, range(1,10)))
    
    #for anyone else who bats later in the game, assign 'PH' (pinch hitter) to their batting order slot
    other_batters = list(set(all_batters) - set(first_9_batters))
    if len(other_batters) > 0:
        for batter in other_batters:
            batting_order_map[batter] = 'PH'
    try:
        game_df['batting_order_slot'] = game_df['batter'].apply(lambda x: batting_order_map[x])    
    except KeyError:
        game_df = None
        return game_df
    
    game_df['pitcher_AB'] = game_df['batter'].apply(lambda x: True if x in pitcher_list else False)
    game_df['batting_order_slot'] = game_df['batting_order_slot'].where(game_df['pitcher_AB'] == False, other='pitcher')
    return game_df

def get_pitch_tendencies(pitcher_df):
    #assign the normalized value counts for this pitchers pitch types to a dictionary
    pitcher_tendencies_overall = pitcher_df['pitch_cat'].value_counts(normalize=True).to_dict()

    #initialize empty dict for count categories tendencies
    pitcher_tendencies_by_count = {}
    
    #loop over each count category and get the pitchers tendencies and add to the dict
    for cat in pitcher_df['count_cat'].unique().tolist():
        subset = pitcher_df[pitcher_df['count_cat'] == cat]
        pitcher_tendencies_by_count[cat] = subset['pitch_cat'].value_counts(normalize=True).to_dict()
    return pitcher_tendencies_overall, pitcher_tendencies_by_count

def make_game_pitchcount_and_trailing_pitch_features_and_batting_order(pitcher_df, pitcher_list):
    df = pitcher_df.copy()
    all_games = []
    
    print('#pitches in df before: ' + str(len(df)))
    
    pitcher_tendencies_overall, pitcher_tendencies_by_count = get_pitch_tendencies(df)
    games = df['game_pk'].unique().tolist()
    
    for game in games:
        
        #take the first game and make the pitch count feature
        game_df = df[df['game_pk'] == game].copy()
        game_df['pitch_count'] = range(1, game_df.shape[0] + 1)
    
        #make the L1_pitch type feature:
        game_df['L1_pitch_type'] = game_df['pitch_cat'].shift(periods=1)
        game_df['L1_pitch_result'] = game_df['type'].shift(periods=1)
        game_df['L1_pitch_result'] = game_df['L1_pitch_result'].replace({np.nan:'first pitch'})
        game_df['L1_pitch_zone'] = game_df['zone'].shift(periods=1)
        game_df['L1_ball_high'] = game_df['ball_high'].shift(periods=1)
        game_df['L1_ball_low'] = game_df['ball_low'].shift(periods=1)
        game_df['L1_ball_left'] = game_df['ball_left'].shift(periods=1)
        game_df['L1_ball_right'] = game_df['ball_right'].shift(periods=1)
        
        game_df[['L1_pitch_zone', 'L1_ball_high', 'L1_ball_low', 'L1_ball_left', 'L1_ball_right']] = game_df[['L1_pitch_zone', 'L1_ball_high', 'L1_ball_low', 'L1_ball_left', 'L1_ball_right']].fillna(-1)
        #game_df['L1_pitch_zone'] = game_df['L1_pitch_zone'].fillna(-1)

        #overall strike % (to fill in for first 5 pitches L5_strike_perc)
        overall_strike_perc = df['type'].value_counts(normalize=True)['S'] * 100

        #make the trailing 5 pitches:
        for index, row in game_df.iterrows():
            #fill NaNs for L1_pitch using same method as when pitch_type was missing
            if row['pitch_count'] == 1:
                random_pitch = random.choices(population=list(pitcher_tendencies_overall.keys()), 
                                   weights=list(pitcher_tendencies_overall.values()), 
                                   k=1)[0]
                game_df.at[index, 'L1_pitch_type'] = random_pitch

            #for the first 5 rows, use overall pitcher tendencies    
            if row['pitch_count'] < 6:
                #fill with overall tendencies
                for pitch in list(pitcher_tendencies_overall.keys()):
                    feature = 'L5_' + pitch + '_perc'
                    game_df.at[index, feature] = pitcher_tendencies_overall[pitch] * 100
                    feature = 'L15_' + pitch + '_perc'
                    game_df.at[index, feature] = pitcher_tendencies_overall[pitch] * 100
                #strike %
                game_df.at[index, 'L5_strike_perc'] = overall_strike_perc
                game_df.at[index, 'L15_strike_perc'] = overall_strike_perc

            else:
                current_pitch = game_df.at[index, 'pitch_count']
                #make a subset of the prev 5 pitches
                subset = game_df[(game_df['pitch_count'] > current_pitch - 6) & (game_df['pitch_count'] < current_pitch)]
                #grab the value count percentages for the last 5 pitches
                subset_percentages = subset['pitch_cat'].value_counts(normalize=True).to_dict()
                try:
                    L5_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                except KeyError:
                    L5_strike_perc = 0

                game_df.at[index, 'L5_strike_perc'] = L5_strike_perc

                #iterate over all possible pitch types this pitcher throws:
                for pitch in list(pitcher_tendencies_overall.keys()):
                    feature = 'L5_' + pitch + '_perc'
                    #if he has thrown that pitch type in last 5
                    try:
                        game_df.at[index, feature] = subset_percentages[pitch] * 100
                    #except for when he hasnt thrown that type in last 5
                    except:
                        game_df.at[index, feature] = 0


                if row['pitch_count'] < 16:
                    #make a subset of the prev 15 pitches
                    subset = game_df[(game_df['pitch_count'] < current_pitch)]
                    #grab the value count percentages for the last 15 pitches
                    subset_percentages = subset['pitch_cat'].value_counts(normalize=True).to_dict()
                    try:
                        L15_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                    except KeyError:
                        L15_strike_perc = 0

                    game_df.at[index, 'L15_strike_perc'] = L15_strike_perc

                    #iterate over all possible pitch types this pitcher throws:
                    for pitch in list(pitcher_tendencies_overall.keys()):
                        feature = 'L15_' + pitch + '_perc'
                        #if he has thrown that pitch type in last 15
                        try:
                            game_df.at[index, feature] = subset_percentages[pitch] * 100
                        #except for when he hasnt thrown that type in last 5
                        except:
                            game_df.at[index, feature] = 0
                else:
                    #make a subset of the prev 15 pitches
                    subset = game_df[(game_df['pitch_count'] > current_pitch - 16) & (game_df['pitch_count'] < current_pitch)]
                    #grab the value count percentages for the last 5 pitches
                    subset_percentages = subset['pitch_cat'].value_counts(normalize=True).to_dict()
                    try:
                        L15_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                    except KeyError:
                        L15_strike_perc = 0

                    game_df.at[index, 'L15_strike_perc'] = L15_strike_perc

                    #iterate over all possible pitch types this pitcher throws:
                    for pitch in list(pitcher_tendencies_overall.keys()):
                        feature = 'L15_' + pitch + '_perc'
                        #if he has thrown that pitch type in last 5
                        try:
                            game_df.at[index, feature] = subset_percentages[pitch] * 100
                        #except for when he hasnt thrown that type in last 5
                        except:
                            game_df.at[index, feature] = 0
                            
        #apply the battting order features to the game:                
        game_df = make_game_batting_order(game_df)
    
        all_games.append(game_df)
    
    new_df = pd.concat(all_games).sort_values(by=['game_date', 'game_pk', 'at_bat_number', 'pitch_number'])
    
    print('# pitches in df after: ' + str(len(new_df)))
    
    return new_df

def make_prev_ab_walk_basehit_run_and_homerun_features(pitcher_df):
    
    all_games = []
    #iterate over each game
    for game in pitcher_df['game_pk'].unique():
        #make subset df for that game
        game_df = pitcher_df[pitcher_df['game_pk'] == game].copy()
        #initialize columns to False:
        game_df['prev_ab_run_scored'] = False
        game_df['prev_ab_homerun'] = False
        game_df['prev_ab_walk'] = False
        game_df['prev_ab_basehit'] = False
        game_df['prev_ab_strikeout'] = False
        
        #this gets the 
        at_bats = game_df['at_bat_number'].sort_values().unique()
        
        #initialize empty dicts
        run_scored = []
        homeruns = []
        walks = []
        basehits = []
        strikeouts = []
        
        walks = ['walk', 'hit_by_pitch']
        basehits = ['single', 'double', 'triple', 'home_run']
        
        #starting w/ 2nd AB, iterate thru to the end of the at_bats:
        for ab in at_bats[2:]:
            #get the index for the last pitch of the prev AB
            prev_ab_last_pitch_index = game_df[game_df['at_bat_number'] == ab-1]['pitch_number'].index.max()
            #check if the last pitch resulted in a walk or hit by pitch:
            if game_df.loc[prev_ab_last_pitch_index]['events'] in walks:
                #if so, add an entry
                walks.append(ab)
            #check if last pitch gave up a basehit:
            elif game_df.loc[prev_ab_last_pitch_index]['events'] in basehits:
                basehits.append(ab)
            elif game_df.loc[prev_ab_last_pitch_index]['events'] == 'strikeout':
                strikeouts.append(ab)
            
            #to check if prev AB resulted in a run scoring: compare score before and after the AB
            prev_score = game_df[game_df['at_bat_number'] == ab-1]['bat_score'].values[0]
            current_score = game_df[game_df['at_bat_number'] == ab]['bat_score'].values[0]
            
            if current_score > prev_score:
                run_scored.append(ab)
                 
                #check if last AB gave up a homerun:
                if game_df.loc[prev_ab_last_pitch_index]['events'] == 'home_run':
                    homeruns.append(ab)
                    
        #iterate over each at_bat, and add the features to the df where appropriate
        for ab in at_bats:
            idx = game_df[game_df['at_bat_number'] == ab].index
            if ab in walks:
                game_df.at[idx, 'prev_ab_walk'] = True
            elif ab in basehits:
                game_df.at[idx, 'prev_ab_basehit'] = True
            elif ab in strikeouts:
                game_df.at[idx, 'prev_ab_strikeout'] = True
            if ab in run_scored:
                game_df.at[idx, 'prev_ab_run_scored'] = True
                if ab in homeruns:
                    game_df.at[idx, 'prev_ab_homerun'] = True
        all_games.append(game_df)
        
    return pd.concat(all_games).sort_values(by=['game_date', 'game_pk', 'pitch_count'])

In [83]:
%%time
pitcher_df = make_game_pitchcount_and_trailing_pitch_features_and_batting_order(pitcher_df, pitcher_list)

#pitches in df before: 5827
# pitches in df after: 5827
CPU times: user 1min 3s, sys: 12.9 ms, total: 1min 3s
Wall time: 1min 3s


In [84]:
%%time
pitcher_df = make_prev_ab_walk_basehit_run_and_homerun_features(pitcher_df)

CPU times: user 17.5 s, sys: 16.9 ms, total: 17.5 s
Wall time: 17.5 s


In [0]:
#pitcher_df[['game_pk','at_bat_number','bat_score','events', 'prev_ab_run_scored', 'prev_ab_homerun', 'prev_ab_walk', 'prev_ab_basehit', 'prev_ab_strikeout']].head(10)

In [86]:
cols = ['pitch_count', 'pitch_type', 'type', 'L1_pitch_type', 'L1_pitch_result', 'L1_pitch_zone','batting_order_slot']	

for col in pitcher_df.columns:
    if col[-4:] == 'perc':
        if col[:2]=='L5':
            cols.append(col)
        if col[:3]=='L15':
            cols.append(col)
        
pitcher_df[cols].head(20)

,pitch_count,pitch_type,type,L1_pitch_type,L1_pitch_result,L1_pitch_zone,batting_order_slot,L5_fastball_perc,L15_fastball_perc,L5_breaking_perc,L15_breaking_perc,L5_offspeed_perc,L15_offspeed_perc,L5_strike_perc,L15_strike_perc
1176,1,FF,B,breaking,first pitch,-1.0,1,51.03827,51.038270,45.752531,45.752531,3.209199,3.209199,47.503003,47.503003
1177,2,FF,B,fastball,B,11.0,1,51.03827,51.038270,45.752531,45.752531,3.209199,3.209199,47.503003,47.503003
1178,3,FF,S,fastball,B,14.0,1,51.03827,51.038270,45.752531,45.752531,3.209199,3.209199,47.503003,47.503003
1179,4,FF,S,fastball,S,1.0,1,51.03827,51.038270,45.752531,45.752531,3.209199,3.209199,47.503003,47.503003
1180,5,FF,S,fastball,S,7.0,1,51.03827,51.038270,45.752531,45.752531,3.209199,3.209199,47.503003,47.503003
1181,6,FT,S,fastball,S,11.0,2,100.00000,100.000000,0.000000,0.000000,0.000000,0.000000,60.000000,60.000000
1182,7,FF,S,fastball,S,14.0,2,100.00000,100.000000,0.000000,0.000000,0.000000,0.000000,80.000000,66.666667
1183,8,FF,B,fastball,S,12.0,2,100.00000,100.000000,0.000000,0.000000,0.000000,0.000000,100.000000,71.428571
1184,9,SL,B,fastball,B,11.0,2,100.00000,100.000000,0.000000,0.000000,0.000000,0.000000,80.000000,62.500000
1185,10,FF,X,breaking,B,13.0,2,80.00000,88.888889,20.000000,11.111111,0.000000,0.000000,60.000000,55.555556


In [87]:
pitcher_df[cols].tail(20)

,pitch_count,pitch_type,type,L1_pitch_type,L1_pitch_result,L1_pitch_zone,batting_order_slot,L5_fastball_perc,L15_fastball_perc,L5_breaking_perc,L15_breaking_perc,L5_offspeed_perc,L15_offspeed_perc,L5_strike_perc,L15_strike_perc
1306256,85,FT,B,fastball,B,11.0,4,60.0,40.000000,20.0,46.666667,20.0,13.333333,60.0,60.000000
1306257,86,SL,X,fastball,B,13.0,4,80.0,40.000000,20.0,46.666667,0.0,13.333333,40.0,53.333333
1306271,87,SL,S,breaking,X,4.0,5,60.0,40.000000,40.0,46.666667,0.0,13.333333,20.0,46.666667
1306272,88,CH,B,breaking,S,12.0,5,60.0,40.000000,40.0,46.666667,0.0,13.333333,40.0,46.666667
1306273,89,FT,S,offspeed,B,14.0,5,40.0,40.000000,40.0,40.000000,20.0,20.000000,20.0,46.666667
1306274,90,SL,B,fastball,S,14.0,5,40.0,46.666667,40.0,40.000000,20.0,13.333333,40.0,53.333333
1306275,91,FT,S,breaking,B,13.0,5,20.0,46.666667,60.0,40.000000,20.0,13.333333,40.0,53.333333
1306276,92,SL,S,fastball,S,5.0,5,40.0,46.666667,40.0,40.000000,20.0,13.333333,60.0,53.333333
1306277,93,FT,S,breaking,S,8.0,6,40.0,46.666667,40.0,40.000000,20.0,13.333333,60.0,53.333333
1306278,94,SL,B,fastball,S,14.0,6,60.0,53.333333,40.0,33.333333,0.0,13.333333,80.0,53.333333


## Fill Batting Scouting Report NaNs

#### Load in the batting_order_slot map pickle

In [0]:
batting_order_slot_map = pickle.load(open("/content/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/batting_order_slot_map.pkl", "rb"))

In [0]:
#batting_order_slot_map

In [0]:
batter_cols = ['fastball_perc_faced','fastball_chase_perc','fastball_bip_swung_perc', 'fastball_taken_strike_perc',
               'fastball_est_woba', 'fastball_babip', 'fastball_iso_value', 'breaking_perc_faced', 'breaking_chase_perc',
               'breaking_bip_swung_perc', 'breaking_taken_strike_perc', 'breaking_est_woba', 'breaking_babip', 
               'breaking_iso_value', 'offspeed_perc_faced', 'offspeed_chase_perc', 'offspeed_bip_swung_perc',
               'offspeed_taken_strike_perc', 'offspeed_est_woba', 'offspeed_babip', 'offspeed_iso_value',
               'pitchout_perc_faced']

def fill_batting_nans(pitcher_df, batting_order_slot_map):
    df = pitcher_df.copy()
    for slot in df['batting_order_slot'].unique().tolist():
        subset = df[df['batting_order_slot'] == slot].copy()
        df = df.drop(subset.index)
        for col in batter_cols:
            subset[col] = subset[col].fillna(batting_order_slot_map[slot][col])
        df = pd.concat([df, subset])
        print('finished w/ slot: ' + str(slot))
    df = df.sort_values(by=['game_date', 'game_pk', 'pitch_count'])
    return df

In [91]:
%%time
pitcher_df = fill_batting_nans(pitcher_df, batting_order_slot_map)

finished w/ slot: 1
finished w/ slot: 2
finished w/ slot: 3
finished w/ slot: 4
finished w/ slot: 5
finished w/ slot: 6
finished w/ slot: 7
finished w/ slot: 8
finished w/ slot: pitcher
finished w/ slot: PH
finished w/ slot: 9
CPU times: user 484 ms, sys: 6.02 ms, total: 490 ms
Wall time: 494 ms


## Add Pitcher / Batter Matchup History Report

In [0]:
def add_pb_matchup_priors(pitcher_df, pitcher_df17, start_dates, end_dates):
    df = pd.concat([pitcher_df, pitcher_df17], sort=False)
    
    #initialize empty list to store dfs (concat them together later)
    df_list = []
    
    #iterate over each period
    for i in range(len(start_dates)):
        
        #make the prior and current dfs:
        prior_df = df[df['game_date'] < start_dates[i]]
        current_df = df[(df['game_date'] >= start_dates[i]) & (df['game_date'] <= end_dates[i])]
        
        #get all the pitch_types this pitcher has thrown in the past:
        pitch_types = prior_df['pitch_cat'].unique().tolist()
        
        try:
            pitch_types.remove('PO')
        except:
            pass
        print(pitch_types)
        
        #get a list of the batters in the current_df
        current_batters = current_df['batter'].unique().tolist()
        
        batters_dict = {}
        
        current_df_list = []
        
        for batter in current_batters:
            batter_df_list = []
            
            #first use subset from prior df
            batter_subset = prior_df[prior_df['batter'] == batter].copy()
            #if pitcher has never faced this batter before:
            if batter_subset.empty:
                #get the left or right handedness of the batter
                stand = current_df[current_df['batter'] == batter]['stand'].values[0]
                #use overall prior tendencies vs left or right handed hitters
                overall, by_count = get_pitch_tendencies(prior_df[prior_df['stand'] == stand])
            else:
                overall, by_count = get_pitch_tendencies(batter_subset)
            batters_dict[batter] = by_count
            
            #now use subset of current_df where batter=batter
            batter_subset = current_df[current_df['batter'] == batter].copy()
            #iterate over the different count_cat types:
            for count_cat in ['ahead', 'behind', 'neutral']:
                count_subset = batter_subset[batter_subset['count_cat'] == count_cat].copy()
                if count_subset.empty:
                    continue
                else:
                    for pitch in pitch_types:
                        try:
                            count_subset['PB_'+pitch] = batters_dict[batter][count_cat][pitch] * 100
                        except KeyError:
                            count_subset['PB_'+pitch] = 0
                            
                current_df_list.append(count_subset)
        
        current_df = pd.concat(current_df_list, sort=False)
        df_list.append(current_df)
                    
    new_df = pd.concat(df_list, sort=False).sort_values(by=['game_date', 'game_pk', 'pitch_count'])
    return new_df

In [93]:
%%time
pitcher_df = add_pb_matchup_priors(pitcher_df, pitcher_df17, start_dates, end_dates)

['fastball', 'breaking', 'offspeed']
['fastball', 'breaking', 'offspeed']
['fastball', 'breaking', 'offspeed']
['fastball', 'breaking', 'offspeed']
['fastball', 'breaking', 'offspeed']
['fastball', 'breaking', 'offspeed']
['fastball', 'breaking', 'offspeed']
['fastball', 'breaking', 'offspeed']
['fastball', 'breaking', 'offspeed']
['fastball', 'breaking', 'offspeed']
['fastball', 'breaking', 'offspeed']
CPU times: user 25.4 s, sys: 35.1 ms, total: 25.4 s
Wall time: 25.5 s


In [94]:
pitcher_df17.pitch_type.value_counts()

SL    1161
FF     841
FT     799
CH     283
Name: pitch_type, dtype: int64

In [95]:
pitcher_df.pitch_type.value_counts()

SL    2273
FT    1812
FF    1162
CU     393
CH     187
Name: pitch_type, dtype: int64

In [96]:
pitcher_df17.pitch_cat.value_counts(normalize=True)

fastball    0.531777
breaking    0.376459
offspeed    0.091764
Name: pitch_cat, dtype: float64

In [97]:
pitcher_df.pitch_cat.value_counts(normalize=True)

fastball    0.510383
breaking    0.457525
offspeed    0.032092
Name: pitch_cat, dtype: float64

In [0]:
#test[['pb_FF', 'pb_CH','pb_CU','pb_SL','pb_FT','pb_FC']]

In [99]:
pitcher_df.isna().sum()

pitch_type                            0
game_date                             0
index                                 0
release_speed                         3
release_pos_x                         4
release_pos_z                         4
batter                                0
pitcher                               0
zone                                  4
hit_location                       4480
game_year                             0
pfx_x                                 4
pfx_z                                 4
plate_x                               4
plate_z                               4
on_3b                                 0
on_2b                                 0
on_1b                                 0
outs_when_up                          0
inning                                0
hc_x                               4889
hc_y                               4889
fielder_2                             0
vx0                                   4
vy0                                   4


## Choose features to use in model input vector, re-cast datatypes, encode categorical variables (onehot and binary), scale appropriately (Standard vs MinMax vs Robust), and check for any remaining NaNs and choose imputation strategy

#### Train/test Split (IMO use April/2017 - end of June/2019 as Train, and July/August 2019 as Test)

In [0]:
#Todo

In [101]:
pitcher_df.columns.tolist()

['pitch_type',
 'game_date',
 'index',
 'release_speed',
 'release_pos_x',
 'release_pos_z',
 'batter',
 'pitcher',
 'zone',
 'hit_location',
 'game_year',
 'pfx_x',
 'pfx_z',
 'plate_x',
 'plate_z',
 'on_3b',
 'on_2b',
 'on_1b',
 'outs_when_up',
 'inning',
 'hc_x',
 'hc_y',
 'fielder_2',
 'vx0',
 'vy0',
 'vz0',
 'ax',
 'ay',
 'az',
 'sz_top',
 'sz_bot',
 'hit_distance_sc',
 'launch_speed',
 'launch_angle',
 'effective_speed',
 'release_spin_rate',
 'release_extension',
 'game_pk',
 'release_pos_y',
 'estimated_ba_using_speedangle',
 'estimated_woba_using_speedangle',
 'woba_value',
 'woba_denom',
 'babip_value',
 'iso_value',
 'launch_speed_angle',
 'at_bat_number',
 'pitch_number',
 'home_score',
 'away_score',
 'bat_score',
 'fld_score',
 'post_away_score',
 'post_home_score',
 'post_bat_score',
 'post_fld_score',
 'stand',
 'p_throws',
 'home_team',
 'away_team',
 'type',
 'bb_type',
 'inning_topbot',
 'sv_id',
 'if_fielding_alignment',
 'score_diff',
 'on_1b_id',
 'ball_high',
 'b

In [0]:
input_vector = ['game_date', 'batter', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up',
                'inning','fielder_2', 'sz_top', 'sz_bot', 'game_pk','at_bat_number',
                'pitch_number', 'bat_score', 'fld_score', 'stand', 'p_throws',
                'inning_topbot', 'if_fielding_alignment','score_diff', 'on_1b_id',
 
                
                
                'ball_high',
 'ball_low',
 'ball_left',
 'ball_right',
 'bases_loaded',
 'batter_swung',
 'in_strikezone',
 'chased',
 'fastball_perc_faced',
 'fastball_chase_perc',
 'fastball_bip_swung_perc',
 'fastball_taken_strike_perc',
 'fastball_est_woba',
 'fastball_babip',
 'fastball_iso_value',
 'breaking_perc_faced',
 'breaking_chase_perc',
 'breaking_bip_swung_perc',
 'breaking_taken_strike_perc',
 'breaking_est_woba',
 'breaking_babip',
 'breaking_iso_value',
 'offspeed_perc_faced',
 'offspeed_chase_perc',
 'offspeed_bip_swung_perc',
 'offspeed_taken_strike_perc',
 'offspeed_est_woba',
 'offspeed_babip',
 'offspeed_iso_value',
 'pitchout_perc_faced',
 'balls',
 'strikes',
 'player_name',
 'events',
 'description',
 'des',
 'pitch_name',
 'of_fielding_alignment',
 '_count',
 'count_cat',
 'pitch_cat',
 'overall_breaking_perc',
 'count_cat_breaking_perc',
 'overall_fastball_perc',
 'count_cat_fastball_perc',
 'overall_offspeed_perc',
 'count_cat_offspeed_perc',
 'pitch_count',
 'L1_pitch_type',
 'L1_pitch_result',
 'L1_pitch_zone',
 'L5_fastball_perc',
 'L15_fastball_perc',
 'L5_breaking_perc',
 'L15_breaking_perc',
 'L5_offspeed_perc',
 'L15_offspeed_perc',
 'L5_strike_perc',
 'L15_strike_perc',
 'batting_order_slot',
 'pitcher_AB',
 'prev_ab_run_scored',
 'prev_ab_homerun',
 'prev_ab_walk',
 'prev_ab_basehit',
 'prev_ab_strikeout',
 'PB_fastball',
 'PB_breaking',
 'PB_offspeed']

In [103]:
pitcher_df.head()

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat,overall_breaking_perc,count_cat_breaking_perc,overall_fastball_perc,count_cat_fastball_perc,overall_offspeed_perc,count_cat_offspeed_perc,pitch_count,L1_pitch_type,L1_pitch_result,L1_pitch_zone,L1_ball_high,L1_ball_low,L1_ball_left,L1_ball_right,L5_fastball_perc,L15_fastball_perc,L5_breaking_perc,L15_breaking_perc,L5_offspeed_perc,L15_offspeed_perc,L5_strike_perc,L15_strike_perc,batting_order_slot,pitcher_AB,prev_ab_run_scored,prev_ab_homerun,prev_ab_walk,prev_ab_basehit,prev_ab_strikeout,PB_fastball,PB_breaking,PB_offspeed
1176,FF,2018-03-29,22960,91.800003,2.2178,6.3667,453568.0,571578.0,11.0,NaN,2018.0,0.5639,1.0993,-0.6422,4.0275,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488671.0,-8.4965,-133.442596,-2.4342,8.6602,29.870899,-18.6679,3.4124,1.6428,NaN,NaN,NaN,91.962997,2216.0,6.539,529410.0,53.963501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,L,L,ARI,COL,B,NaN,Top,180330_021111,Standard,0.0,NaN,True,False,False,False,0,0,0,0,65.776695,29.307116,39.467312,38.118214,0.40704,0.246085,0.221477,24.895977,36.879433,35.422344,28.474577,0.471,0.221649,0.293814,9.327323,37.572254,40.277779,17.708334,0.459517,0.28,0.226667,0.0,0,0,Patrick Corbin,NaN,ball,NaN,4-Seam Fastball,Standard,00,neutral,fastball,52.143845,53.600000,45.919779,44.400000,1.936376,2.000000,1.0,breaking,first pitch,-1.0,-1,-1,-1,-1,51.03827,51.03827,45.752531,45.752531,3.209199,3.209199,47.503003,47.503003,1,False,False,False,False,False,False,40.000000,60.000000,0.000000
1177,FF,2018-03-29,22949,91.900002,2.2893,6.2875,453568.0,571578.0,14.0,NaN,2018.0,0.7047,1.1697,0.1060,1.4088,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488671.0,-7.0477,-133.367096,-8.9738,9.9698,30.138700,-16.4660,3.3972,1.6277,NaN,NaN,NaN,91.573997,2298.0,6.360,529410.0,54.141998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,L,L,ARI,COL,B,NaN,Top,180330_021131,Standard,0.0,NaN,False,True,False,False,0,0,0,0,65.776695,29.307116,39.467312,38.118214,0.40704,0.246085,0.221477,24.895977,36.879433,35.422344,28.474577,0.471,0.221649,0.293814,9.327323,37.572254,40.277779,17.708334,0.459517,0.28,0.226667,0.0,1,0,Patrick Corbin,NaN,ball,NaN,4-Seam Fastball,Standard,10,behind,fastball,52.143845,37.113402,45.919779,61.855670,1.936376,1.030928,2.0,fastball,B,11.0,True,False,False,False,51.03827,51.03827,45.752531,45.752531,3.209199,3.209199,47.503003,47.503003,1,False,False,False,False,False,False,33.333333,66.666667,0.000000
1178,FF,2018-03-29,22932,92.599998,2.2268,6.3818,453568.0,571578.0,1.0,NaN,2018.0,0.3072,1.3006,-0.6351,3.0080,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488671.0,-7.9257,-134.451004,-5.5301,5.5154,31.222099,-15.5747,3.1735,1.2750,NaN,NaN,NaN,91.8